In [1]:
#! venv/bin/ python3
# *-encoding: utf8 -*

import requests as rqs
from bs4 import BeautifulSoup
import datetime
import time
import pandas as pd


def unixTimeStampConvert(num):
    convert = datetime.datetime.fromtimestamp(int(num)).strftime('%Y-%m-%d %H:%M:%S')
    return convert

In [2]:
# CONEXION AL SERVIDOR (SITIO WEB), Y REQUEST (PETICION).
# **********************************************************
# A que sitio me quiero conectar, parametro y URI
myurl = "https://votaciones.hcdn.gob.ar"
# Los siguientes datos los saque de la misma pagina, mirando el codigo fuente para ver
# como enviaba las peticiones y cual debia enviar para obtener todos los datos =)
formQuery = {'anoSearch': -1, 'txtSearch': ''}
endPoint = "https://votaciones.hcdn.gob.ar/votaciones/search"

# Peticion al servidor
r = rqs.post(endPoint, data=formQuery)

# Estado de la conexion
status = r.status_code

if status == 200:
    print(f'Conectando a {myurl}')
    print(f'Estado de la conexion :: {status}')
else:
    print('La conexion no se establecio correctamente')
    print(f'Codigo estado : {status}')
    quit()
# **********************************************************

Conectando a https://votaciones.hcdn.gob.ar
Estado de la conexion :: 200


In [3]:
# Antes de programar el scraper deberiamos saber bien cuanl es el objeto html que contiene la informacion que
# queremos por ejemplo, cual es el tag que contiene la info que queremos.
# podemos identificar el objeto con la informacion a traves del nombre del Tag, del selctor css, "class", del id, etc

# El elemento que me interesa tiene la clase "row-acta"
clase = "row-acta"

# Creo el objeto BeautifulSoup
soup = BeautifulSoup(r.content, 'html.parser')

# El tipo de dato es un result set de beautiful soup, es un iterable, algo asi como una lista
datos = soup.find_all("tr", class_=clase)

In [4]:
# EXTRACCION DE LAS URLS DONDE ESTAN LOS DETALLES DE CADA VOTACION
# **********************************************************
urlActas = list()
for t in datos:
    # Columnas
    cols = t.find_all('td')
    idActa = t['id']  # Seria Clave principal
    date = unixTimeStampConvert(t['data-date']) # Tengo que convertir el formato de la fecha en human friendly =)
    # Me voy a guardar los datos que extraje en una lista de tuplas
    urlActas.append((myurl + cols[4].find('a')['href'], idActa, date))
# **********************************************************

In [5]:
# RECORRIDO DE LAS ACTAS Y SCRAPING DE LOS DETALLES - ACTAS Y VOTACION
# **********************************************************
actas = list()
votos = list()
count = 1
ctrlCount = 0
for a in urlActas:
    #Conexion a cada Acta # *********************************
    r = rqs.get(a[0])
    soup = BeautifulSoup(r.content, 'html.parser')
    
    # idActa = a[1] este es el id que figura en el sitio, pero voy a usar otro
    dateActa = a[2]
    urlActa = a[0]

    # VAmos a recuperar estos datos -> Período 123 - Reunión 40 - Acta 31
    try:
        text = soup.find('h5').text.split('-')
        chequePoint_1 = True
    except:
        chequePoint_1 = False
        text = ['- None', '- None', '- None']
        
    periodo = text[0].split()[1]
    reunion = text[1].split()[1]
    numeroActa = text[2].split()[1]
    idSite = a[1]
    idActa = '{}-{}-{}'.format(periodo, reunion, numeroActa)

    # Acta # *********************************
    try:
        div = soup.find('div', class_='white-box')
        
        h3 = div.find_all('h3')
        h4 = div.find_all('h4')

        titulo = h4[0].text.strip().lower()
        presidente = h4[1].find('b').text
        resolucion = h3[0].text
        chequePoint_2 = True
        
    except:
        chequePoint_2 = False
        titulo = None
        presidente = None
        resolucion = None

    if not chequePoint_1 :
        if not chequePoint_2:
            continue
        
    actas.append({
            'idActa':idActa,
            'fechaActa': dateActa,
            'periodo': periodo,
            'reunion': reunion,
            'numeroActa': numeroActa,
            'titulo': titulo,
            'presidente': presidente,
            'resolucion': resolucion,
            'urlActa': urlActa,
            'idSite': idSite
        }
    )
    
    
    # Votacion # *********************************
    tabla = soup.find('table', id='myTable')
    rows = tabla.find_all('tr')
#     count = 1
    for r in rows:
        cols = [x for x in r.find_all('td')]
        if cols:
            try:
                idDip = cols[0].find('div')['id'].split('-')[1]
            except TypeError:
                pass
            
            voto = cols[4].text.strip().lower()
            dichos = cols[5].text.strip().lower()
            
            votos.append({
                    'idActa': idActa,
                    'idVoto': count,
                    'voto': voto,
                    'numeroActa': numeroActa,
                    'dip': idDip,
                    'dichos': dichos               
            })
            count += 1
            
    ctrlCount += 1
            
#     if not votos:
#         continue
#     else:
#         votaciones.append(votos)
        
#     time.sleep(1)
# **********************************************************

In [6]:
urlActa

'https://votaciones.hcdn.gob.ar/votacion/mocion-diputado-ruckauf-tratamiento-proscripcion-armas-america-latina-caribe-tratado-tlatelolco/3622'

In [7]:
idActa

'111-9-2'

*******************************

In [8]:
# CONEXION A LA PAGINA DE DONDE VAMOS A SACAR LOS DATOS DE TODOS LOS DIPUTADOS
# Los datos de cada diputado estan en una pagina diferente, por eso voy a hacer nuevas peticiones
# **********************************************************
myurl = "https://votaciones.hcdn.gob.ar/estadisticas/home"
# Los siguientes datos los saque de la misma pagina, mirando el codigo fuente para ver
# como enviaba las peticiones y cual debia enviar para obtener todos los datos =)
formQuery = {'anoSearchEstadistica': -1}
endPoint = "https://votaciones.hcdn.gob.ar/estadisticas/search"

# Peticion al servidor
r = rqs.post(endPoint, data=formQuery)

# Estado de la conexion
status = r.status_code

if status == 200:
    print(f'Conectando a {myurl}')
    print(f'Estado de la conexion :: {status}')
else:
    print('La conexion no se establecio correctamente')
    print(f'Codigo estado : {status}')
    quit()
# **********************************************************

soup = BeautifulSoup(r.content, 'html.parser')
table = soup.find('table', id='myTable')
rows = table.find_all('tr')

# for r in rows:
#     cols = r.find_all('td')
    

Conectando a https://votaciones.hcdn.gob.ar/estadisticas/home
Estado de la conexion :: 200


In [9]:
myurl = 'https://votaciones.hcdn.gob.ar'

In [10]:
diputados = list()
for r in rows:
    cols = r('td')
    if cols:

        idDip = cols[0].find('div')['id'].split('-')[1]
        urlPerf = myurl + cols[1].find('a')['href']
        fullName = cols[1].text.strip()

        if len(fullName) > 0:
            apellido = fullName.split(',')[0]
            nombre = fullName.split(',')[1]
        else:
            apellido = ''
            nombre = ''

        estado = cols[2].text.strip()
        bloque = cols[3].text.strip()
        provincia = cols[4].text.strip()
        afirmativos = cols[5].text.strip()
        negativos = cols[6].text.strip()
        abstenciones = cols[7].text.strip()
        ausencias = cols[8].text.strip()
        presidencias = cols[9].text.strip()

#         diputados.append({
#             idDip: {
#                 'idDip': idDip,
#                 'urlPerf': urlPerf,
#                 'fullName': fullName,
#                 'apellido': apellido,
#                 'nombre': nombre,
#                 'estado': estado,
#                 'bloque': bloque,
#                 'provincia': provincia,
#                 'afirmativos': afirmativos,
#                 'negativos': negativos,
#                 'abstenciones': abstenciones,
#                 'ausencias': ausencias,
#                 'presidencias': presidencias
#                 }
#             }
#         )
        diputados.append({
                    'idDip': idDip,
                    'urlPerf': urlPerf,
                    'fullName': fullName,
                    'apellido': apellido,
                    'nombre': nombre,
                    'estado': estado,
                    'bloque': bloque,
                    'provincia': provincia,
                    'afirmativos': afirmativos,
                    'negativos': negativos,
                    'abstenciones': abstenciones,
                    'ausencias': ausencias,
                    'presidencias': presidencias
                    }
                )     

In [11]:
# # Convertir la lista de diputados en un Json
# import json as js
# dipJson = {'diputados':diputados}
# dipJson = js.dumps(dipJson, indent=4)

In [12]:
# [list(x .keys())[0] for x in diputados]
diputados_till_25012020 = pd.DataFrame([x for x in diputados])


In [13]:
actas_till_25012020 = pd.DataFrame([x for x in actas])

In [14]:
votos_till_25012020 = pd.DataFrame([x for x in votos]).rename(columns={'dip':'idDip'})

In [15]:
diputados_till_25012020.to_csv('diputados_till_25012020.csv', index=False)
actas_till_25012020.to_csv('actas_till_25012020.csv', index=False)
votos_till_25012020.to_csv('votos_till_25012020.csv', index=False)

In [16]:
# pd.DataFrame([x for x in votos]).rename(columns={'dip':'idDip'}).merge(pd.DataFrame([x for x in diputados]), how='inner', on='idDip')

In [17]:
# pd.DataFrame([x for x in votos]).rename(columns={'dip':'idDip'}).merge(pd.DataFrame([x for x in diputados]), how='inner', on='idDip').merge(pd.DataFrame([x for x in actas]).rename(columns={'idActa':'idVotacion'}), how='inner', on='idVotacion').columns

In [18]:
actas_till_25012020

,idActa,fechaActa,periodo,reunion,numeroActa,titulo,presidente,resolucion,urlActa,idSite
0,137-12-27,2019-12-20 21:07:06,137,12,27,expediente 0016-pe-2019 - o.d. 2 - ley de soli...,"MASSA, Sergio Tomás",AFIRMATIVO,https://votaciones.hcdn.gob.ar/votacion/3959,3959
1,137-12-26,2019-12-20 20:56:06,137,12,26,expediente 0016-pe-2019 - o.d. 2 - ley de soli...,"MASSA, Sergio Tomás",AFIRMATIVO,https://votaciones.hcdn.gob.ar/votacion/3958,3958
2,137-12-25,2019-12-20 20:53:17,137,12,25,expediente 0016-pe-2019 - o.d. 2 - ley de soli...,"MASSA, Sergio Tomás",AFIRMATIVO,https://votaciones.hcdn.gob.ar/votacion/3957,3957
3,137-12-24,2019-12-20 20:47:04,137,12,24,expediente 0016-pe-2019 - o.d. 2 - ley de soli...,"MASSA, Sergio Tomás",AFIRMATIVO,https://votaciones.hcdn.gob.ar/votacion/3956,3956
4,137-12-23,2019-12-20 20:31:03,137,12,23,expediente 0016-pe-2019 - o.d. 2 - ley de soli...,"MASSA, Sergio Tomás",AFIRMATIVO,https://votaciones.hcdn.gob.ar/votacion/3955,3955
...,...,...,...,...,...,...,...,...,...,...
2386,111-10-6,1993-07-01 07:36:00,111,10,6,régimen de defensa del consumidor - capítulo v...,"BROOK, Mario Carlos",AFIRMATIVO,https://votaciones.hcdn.gob.ar/votacion/regime...,3626
2387,111-10-5,1993-07-01 07:24:00,111,10,5,régimen de defensa del consumidor - capítulo v...,"BROOK, Mario Carlos",AFIRMATIVO,https://votaciones.hcdn.gob.ar/votacion/regime...,3625
2388,111-10-2,1993-07-01 05:21:00,111,10,2,moción de orden del diputado spinosa solicitan...,"MARTINEZ, Luis Alberto",NEGATIVO,https://votaciones.hcdn.gob.ar/votacion/mocion...,3624
2389,111-9-3,1993-06-24 05:41:00,111,9,3,moción del diputado spinosa solicitando el tra...,"PIERRI, Alberto Reinaldo",AFIRMATIVO,https://votaciones.hcdn.gob.ar/votacion/mocion...,3623


In [19]:
diputados_till_25012020

,idDip,urlPerf,fullName,apellido,nombre,estado,bloque,provincia,afirmativos,negativos,abstenciones,ausencias,presidencias
0,A50768,https://votaciones.hcdn.gob.ar/diputado/1722,". NO INCORPORADO, Legislador 1",. NO INCORPORADO,Legislador 1,-,Justicialista por Tucumán,Tucumán,-,-,-,27,-
1,A50774,https://votaciones.hcdn.gob.ar/diputado/1719,". NO INCORPORADO, Legislador 2",. NO INCORPORADO,Legislador 2,-,PRO,C.A.B.A.,-,-,-,36,-
2,A4068,https://votaciones.hcdn.gob.ar/diputado/596,"ABALOS, Roberto José",ABALOS,Roberto José,-,Unión Cívica Radical,Santiago del Estero,61,96,6,50,-
3,G344,https://votaciones.hcdn.gob.ar/diputado/1531,"ABALOVICH, Eduardo Antonio",ABALOVICH,Eduardo Antonio,-,Unión Cívica Radical,Santiago del Estero,41,33,4,22,-
4,G309,https://votaciones.hcdn.gob.ar/diputado/1496,"ABAN, Gloria del S.",ABAN,Gloria del S.,-,Justicialista,Salta,58,23,2,17,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751,A51101,https://votaciones.hcdn.gob.ar/diputado/1675,"ZOTTOS, Miguel Andrés Costas",ZOTTOS,Miguel Andrés Costas,En actividad,Justicialista,Salta,418,161,52,366,-
1752,A3629,https://votaciones.hcdn.gob.ar/diputado/1176,"ZUCCARDI, Cristina",ZUCCARDI,Cristina,-,FREPASO,Mendoza,62,24,6,73,-
1753,G297,https://votaciones.hcdn.gob.ar/diputado/1481,"ZUCCARDI de FLAMARIQUE, María",ZUCCARDI de FLAMARIQUE,María,-,Justicialista,Mendoza,52,28,1,83,-
1754,A78,https://votaciones.hcdn.gob.ar/diputado/1177,"ZUÑIGA, Ovidio Octavio",ZUÑIGA,Ovidio Octavio,-,Justicialista,Rio Negro,70,61,7,27,-


In [20]:
votos_till_25012020


,idActa,idVoto,voto,numeroActa,idDip,dichos
0,111-9-2,614231,afirmativo,2,A4674,
1,111-9-2,614232,afirmativo,2,G32,
2,111-9-2,614233,negativo,2,G70,
3,111-9-2,614234,negativo,2,G97,
4,111-9-2,614235,negativo,2,A1186,
...,...,...,...,...,...,...
252,111-9-2,614483,afirmativo,2,G53,
253,111-9-2,614484,ausente,2,A565,
254,111-9-2,614485,ausente,2,G221,
255,111-9-2,614486,ausente,2,A3699,
